In [1]:
from __future__ import print_function

import os
import sys
import argparse
import time
import math

import tensorboard_logger as tb_logger
import torch
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets

from util import TwoCropTransform, AverageMeter
from util import adjust_learning_rate, warmup_learning_rate
from util import set_optimizer, save_model
from networks.resnet_big import SupConResNet,LinearClassifier
from losses import SupConLoss
from adv_train import PGDCons , PGDConsMulti

from main_ce import set_loader
from adv_train import PGDAttack
import torch.optim as optim

2022-11-03 12:29:07.227803: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def set_loader():
    # construct data loader
    mean = (0.4914, 0.4822, 0.4465)
    std = (0.2023, 0.1994, 0.2010)

    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=32, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])


    train_dataset = datasets.CIFAR10(root='./datasets/',
                                        transform=TwoCropTransform(train_transform),
                                        download=True)


    train_sampler = None
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=8, shuffle=(train_sampler is None),
        num_workers=16, pin_memory=True, sampler=train_sampler)

    return train_loader


def set_model():
    model = SupConResNet(name='resnet18')
    criterion = SupConLoss(temperature=0.1)

    # enable synchronized Batch Normalization
    # if opt.syncBN:
    #     model = apex.parallel.convert_syncbn_model(model)

    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:
            model.encoder = torch.nn.DataParallel(model.encoder,device_ids = [0,1])
        model = model.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True

    return model, criterion


In [ ]:
def set_model_linear(opt):
    model = SupConResNet(name=opt.model)
    criterion = torch.nn.CrossEntropyLoss()

    classifier = LinearClassifier(name=opt.model, num_classes=opt.n_cls)

    ckpt = torch.load(opt.ckpt, map_location='cpu')
    state_dict = ckpt['model']

    if torch.cuda.is_available():
        if torch.cuda.device_count() > 1:
            model.encoder = torch.nn.DataParallel(model.encoder, device_ids = [0,1])
        else:
            new_state_dict = {}
            for k, v in state_dict.items():
                k = k.replace("module.", "")
                new_state_dict[k] = v
            state_dict = new_state_dict
        model = model.cuda()
        classifier = classifier.cuda()
        criterion = criterion.cuda()
        cudnn.benchmark = True

        model.load_state_dict(state_dict)

    return model, classifier, criterion


def set_loader_linear(opt):
    # construct data loader
    if opt.dataset == 'cifar10':
        mean = (0.4914, 0.4822, 0.4465)
        std = (0.2023, 0.1994, 0.2010)
    elif opt.dataset == 'cifar100':
        mean = (0.5071, 0.4867, 0.4408)
        std = (0.2675, 0.2565, 0.2761)
    else:
        raise ValueError('dataset not supported: {}'.format(opt.dataset))
    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=32, scale=(0.2, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

    val_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    if opt.dataset == 'cifar10':
        train_dataset = datasets.CIFAR10(root=opt.data_folder,
                                         transform=train_transform,
                                         download=True)
        val_dataset = datasets.CIFAR10(root=opt.data_folder,
                                       train=False,
                                       transform=val_transform)
    elif opt.dataset == 'cifar100':
        train_dataset = datasets.CIFAR100(root=opt.data_folder,
                                          transform=train_transform,
                                          download=True)
        val_dataset = datasets.CIFAR100(root=opt.data_folder,
                                        train=False,
                                        transform=val_transform)
    else:
        raise ValueError(opt.dataset)

    train_sampler = None
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=opt.batch_size, shuffle=(train_sampler is None),
        num_workers=opt.num_workers, pin_memory=True, sampler=train_sampler)
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=256, shuffle=False,
        num_workers=8, pin_memory=True)

    return train_loader, val_loader


def set_optimizer(opt, model):
    optimizer = optim.SGD(model.parameters(),
                          lr=opt.learning_rate,
                          momentum=opt.momentum,
                          weight_decay=opt.weight_decay)
    return optimizer

In [ ]:
train_loader, val_loader = set_loader_linear(opt)

# build model and criterion
model, classifier, criterion = set_model_linear(opt)

# build optimizer
optimizer = set_optimizer(opt, classifier)


test_attack = PGDAttack(model, classifier, eps=8./255., alpha = 2./255., steps=50)

In [3]:
train_loader = set_loader()

# build model and criterion
model, criterion = set_model()

# build optimizer
optimizer = torch.optim.SGD(model.parameters(),
                        lr=0.5,
                        momentum=0.9,
                        weight_decay=1e-4)
                        
atk = PGDCons(model, eps=8./255, alpha=2./225, steps=10, random_start=False)

Files already downloaded and verified


In [4]:
import torchattacks
torchattacks.__version__

'3.3.0'

In [7]:
atk.set_normalization_used(mean = (0.4914, 0.4822, 0.4465), std = (0.2023, 0.1994, 0.2010))

In [4]:
images, labels = next(iter(train_loader))
images = torch.cat([images[0], images[1]], dim=0)
images = images.cuda(non_blocking=True)
labels = labels.cuda(non_blocking=True)
bsz = labels.shape[0]
adv_images = atk(images, labels, loss = criterion)

device:  cuda:0


In [8]:
adv_images = atk(images, labels, loss = criterion)

device:  cuda:0


In [9]:
features = model(adv_images)
f1, f2 = torch.split(features, [bsz, bsz], dim=0) #f1 and f2 -> torch.Size([bsz, 128]
features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1) #torch.Size([bsz, 2 (view), 128(feature dim)])
loss = criterion(features, labels)
loss

tensor(3.6941, device='cuda:0', grad_fn=<MeanBackward0>)

In [6]:
features = model(images)
f1, f2 = torch.split(features, [bsz, bsz], dim=0) #f1 and f2 -> torch.Size([bsz, 128]
features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1) #torch.Size([bsz, 2 (view), 128(feature dim)])
loss = criterion(features, labels)
loss

tensor(3.7763, device='cuda:0', grad_fn=<MeanBackward0>)

In [24]:
features = model(images)
features.shape

torch.Size([16, 128])

In [9]:
multi_atc = PGDConsMulti(model, eps=8./255, alpha=2./225, steps=10, random_start=True)

In [10]:
attacks = multi_atc(images, labels, loss = criterion)

device:  cuda:0


In [11]:
len(attacks)

10

In [37]:
images.shape

torch.Size([16, 3, 32, 32])

In [12]:
attacks[0].shape

torch.Size([16, 3, 32, 32])

In [ ]:
features = model(images)
f1, f2 = torch.split(features, [bsz, bsz], dim=0) #f1 and f2 -> torch.Size([bsz, 128]
features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1) #torch.Size([bsz, 2 (view), 128(feature dim)])
loss = criterion(features, labels)

In [13]:
attacks.append(images)
len(attacks)

11

In [14]:
attacks = torch.cat(attacks, dim=0)

In [16]:
attacks.shape

torch.Size([176, 3, 32, 32])

In [17]:
features = model(attacks)
features.shape

torch.Size([176, 128])

In [18]:
pgd_steps = 10

In [19]:
fs = torch.split(features, [bsz for i in range(2*pgd_steps + 2)], dim=0)

In [20]:
fs[0].shape

torch.Size([8, 128])

In [21]:
features = torch.cat([f.unsqueeze(1) for f in fs], dim=1) #torch.Size([bsz, 2 (view), 128(feature dim)])
loss = criterion(features, labels)
loss

tensor(7.4490, device='cuda:0', grad_fn=<MeanBackward0>)

In [22]:
features.shape

torch.Size([8, 22, 128])